In [8]:
import random
import numpy as numpy
from qiskit import QuantumCircuit
from qiskit_aer import Aer, AerSimulator
from qiskit import transpile

In [9]:
#Primeira etapa: Geração dos bits e bases de Alice
n_qubits = 10

#Alice escolhe aleatóriamente os bits e as bases
alice_bits = [random.randint(0, 1) for _ in range(n_qubits)]
alice_bases = [random.choice(['Z', 'X']) for _ in range(n_qubits)]

In [10]:
#BOb escolhe aleatóriamente as bases de medição
bob_bases = [random.choice(['Z', 'X']) for _ in range(n_qubits)]
bob_results = []

In [11]:
#Criação e execução do circuito quântico
simulator =Aer.get_backend('qasm_simulator')

for i in range(n_qubits):
    qc = QuantumCircuit(1, 1) #um bit quantico e um bit clássico

    #Preparação do estado quântico baseado nos bits e bases de Alice
    if alice_bases[i] == 'Z' :
        if alice_bits[i] == 1:
            qc.x(0)
    else:
        if alice_bits[i] == 0 :
            qc.h(0)
        else :
            qc.x(0)
            qc.h(0)

In [12]:
# Detalhes da preparação do estado: 
# - Ao usar a base Z (base computacional), um bit 0 é codificado como |0⟩ e um bit 1 como |1⟩ (aplicando uma porta X). 
# - Ao usar a base X (base diagonal), um bit 0 é codificado como |+⟩ e um bit 1 como |−⟩. 
# 
# A porta Hadamard (H) é usada para transformar estados de base computacional em estados de base diagonal: 
# H|0⟩ = |+⟩ e H|1⟩ = |−⟩. 
# 
# Essa transformação é crucial em protocolos de distribuição de chaves quânticas (por exemplo, BB84). Se um qubit preparado em uma base 
# for medido em uma base diferente, o resultado da medição se torna aleatório (com uma probabilidade de 50-50), que é uma propriedade chave 
# que garante a segurança do protocolo. 

#Medição do qubit baseado nas bases escolhidas por Bob

# se a base de Bob for X, aplique Hadamard para se preparar para as medições 
  # Observe que há uma chance de 50-50 do qbit colapsar como 0 ou 1 

if bob_bases[i] == 'X' :
    qc.h(0)

qc.measure(0, 0)

circuit = transpile(qc, simulator)
job = simulator.run(circuit)

counts = job.result().get_counts()

measured_bit = int (list(counts.keys())[0])
bob_results.append(measured_bit)



In [13]:
# Etapa de sifting: Alice e Bob comparam suas bases publicamente e mantêm apenas os bits onde as bases coincidem

sifted_key = []
for i, (a_bit, a_base, b_base, b_result) in enumerate(zip(alice_bits, alice_bases, bob_bases, bob_results)):
    if a_base == b_base:
        sifted_key.append((a_bit))
        print(f"Qubit {i}: Basis match ({a_base}). Alice's bit: {a_bit}, Bob's measurement: {b_result}")
    else:
        print(f"Qubit {i}: Basis mismatch (Alice: {a_base}, Bob: {b_base})")

print("\nFinal sifted key:", sifted_key)


Qubit 0: Basis match (X). Alice's bit: 1, Bob's measurement: 1

Final sifted key: [1]


In [14]:
qber = sum(a != b for a, b in zip(sifted_key, sifted_key)) / len(sifted_key)
print(f"QBER: {qber:.2%}")

QBER: 0.00%
